In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
BATCH_SIZE = 64
SEQ_LEN = 128
EPOCHS = 4000
lr = 2e-4
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
EMBEDDING_DIM = 256
NUM_HEADS = 6
NUM_BLOCKS = 6
DROPOUT_RATE = 0.3

torch.manual_seed(69)

In [3]:
with open('data/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
chars = sorted(list(set(text)))
VOCAB_SIZE = len(chars)
# create a mapping from characters to integers
CHAR_TO_INDEX = { ch:i for i,ch in enumerate(chars) }
INDEX_TO_CHAR = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [CHAR_TO_INDEX[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([INDEX_TO_CHAR[i] for i in l]) # decoder: take a list of integers, output a string

In [5]:
data = torch.tensor(encode(text), dtype=torch.long)
pct = int(0.9*len(data))
train_data = data[:pct]
val_data = data[pct:]

In [6]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - SEQ_LEN, (BATCH_SIZE,))
    x = torch.stack([data[i:i+SEQ_LEN] for i in ix])
    y = torch.stack([data[i+1:i+SEQ_LEN+1] for i in ix])
    x, y = x.to(DEVICE), y.to(DEVICE)
    return x, y

In [7]:
class FeedForward(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(EMBEDDING_DIM, 4*EMBEDDING_DIM)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(4*EMBEDDING_DIM, EMBEDDING_DIM)
        self.dropout = nn.Dropout(DROPOUT_RATE)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.dropout(x)
        return x

In [8]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(EMBEDDING_DIM, head_size, bias=False)
        self.query = nn.Linear(EMBEDDING_DIM, head_size, bias=False)
        self.value = nn.Linear(EMBEDDING_DIM, head_size, bias=False)
        self.dropout = nn.Dropout(DROPOUT_RATE)
        self.register_buffer('tril', torch.tril(torch.ones(SEQ_LEN, SEQ_LEN)))

    def forward(self, x):
        B,S,C = x.shape
        k = self.key(x)   # (B,S,C)
        q = self.query(x) # (B,S,C)
        v = self.value(x) # (B,S,C)
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, S, C) @ (B, C, S) -> (B, S, S)
        wei = wei.masked_fill(self.tril[:S, :S] == 0, float('-inf')) # (B, S, S)
        wei = F.softmax(wei, dim=-1) # (B, S, S)
        wei = self.dropout(wei)
        out = wei @ v # (B, S, S) @ (B, S, C) -> (B, S, C)
        return out

In [9]:
class MultipleHead(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.fc = nn.Linear(num_heads*head_size, EMBEDDING_DIM)
        self.dropout = nn.Dropout(DROPOUT_RATE)

    def forward(self, x):
        x = torch.cat([head(x) for head in self.heads], dim=-1)
        x = self.fc(x)
        x = self.dropout(x)
        return x

In [10]:
class Block(nn.Module):
    def __init__(self, num_heads):
        super().__init__()
        head_size = EMBEDDING_DIM // num_heads
        self.ln1 = nn.LayerNorm(EMBEDDING_DIM)
        self.ln2 = nn.LayerNorm(EMBEDDING_DIM)
        self.ff = FeedForward()
        self.mh = MultipleHead(num_heads, head_size)

    def forward(self, x):
        x = x + self.mh(x)
        x = self.ln1(x)
        x = x + self.ff(x)
        x = self.ln2(x)
        return x

In [11]:
class Attention(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(VOCAB_SIZE, EMBEDDING_DIM)
        self.position_embedding_table = nn.Embedding(SEQ_LEN, EMBEDDING_DIM)
        self.blocks = nn.ModuleList([Block(NUM_HEADS) for _ in range(NUM_BLOCKS)])
        self.fc = nn.Linear(EMBEDDING_DIM, VOCAB_SIZE)
        self.ln = nn.LayerNorm(EMBEDDING_DIM)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=DEVICE)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        for block in self.blocks:
            x = block(x)
        x = self.ln(x)
        logits = self.fc(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -SEQ_LEN:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :] # becomes (B, C)
            probs = F.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [12]:
model = Attention()
optimizer = optim.Adam(model.parameters(), lr=lr)
model = model.to(DEVICE)

In [13]:
model.to(DEVICE)
for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1}", end=' ')
    model.train()
    x, y = get_batch("train")
    optimizer.zero_grad()
    y_pred, loss = model(x, y)
    print(f"Training Loss: {loss} -", end=' ')
    loss.backward()
    optimizer.step()
    model.eval()
    x, y = get_batch("val")
    y_pred, loss = model(x, y)
    print(f"Validation Loss: {loss}")

Epoch 1 Training Loss: 4.377954483032227 - Validation Loss: 3.8231306076049805
Epoch 2 Training Loss: 3.847848892211914 - Validation Loss: 3.5939509868621826
Epoch 3 Training Loss: 3.6034343242645264 - Validation Loss: 3.4511916637420654
Epoch 4 Training Loss: 3.4361324310302734 - Validation Loss: 3.3645551204681396
Epoch 5 Training Loss: 3.363511800765991 - Validation Loss: 3.330796718597412
Epoch 6 Training Loss: 3.325648069381714 - Validation Loss: 3.322033405303955
Epoch 7 Training Loss: 3.3063910007476807 - Validation Loss: 3.265164375305176
Epoch 8 Training Loss: 3.2770447731018066 - Validation Loss: 3.2731990814208984
Epoch 9 Training Loss: 3.2694289684295654 - Validation Loss: 3.261064291000366
Epoch 10 Training Loss: 3.261054039001465 - Validation Loss: 3.2031450271606445
Epoch 11 Training Loss: 3.1994142532348633 - Validation Loss: 3.1906580924987793
Epoch 12 Training Loss: 3.18025279045105 - Validation Loss: 3.1926510334014893
Epoch 13 Training Loss: 3.1957931518554688 - Val

In [15]:
# context = torch.zeros((1, 1), dtype=torch.long, device=DEVICE)
context = 'KING'
context = torch.tensor(encode(context), dtype=torch.long, device=DEVICE).unsqueeze(0)
context = context.to(DEVICE)
print(decode(model.generate(context, max_new_tokens=1000)[0].tolist()))

KING EDWARD IV:
And such the most tress of sir, glade
To think my Plantagain, soul have draw for who put thy?

KING RICHARD III:
Hu play thee, thee, our mock?
What, God well men Duke me Edward did Slauged Angelo brathers,
And with do cannot filling eight of it,
For God news, you less against will no do suits
Where contry 'twixt the a father's look.

GLOUCESTER:
Tut this brought death acquestion;
It will was not I was as fit is annow and.

MARIANA:
My lord? Nay, what's he stands?

PETER:
He is thee? which say my brother safe, But black
Which course desceit not now! and he which
pass name, go me day? which is their time, you bruthed
Uquire of misdertal us. Why said him nevil!
Hark it be son? what's we do you?

FLORIZEL:
You call not what dock of Eight-day?

First Servant:
Never that she such of A be think fail subject a dead;
Ot profe I have strumf.

PAULINA:
Mark,
We ars the husband let's loike furming life.
But not set himself in now be dispite,
His every and upon my king your belood;


In [16]:
torch.save(model.state_dict(), 'model.pth')